In [640]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
import pickle
import itertools
import random
#1番目
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

#2番目
#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))

#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_url=[]
false=[]
all_title=[]
text1=[]
for flatclosing_url in flatclosing_urls[50:60]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    all_title.append(title)
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    tagget= soup.find_all('h2')
    text1.append(tagget)
text1_stringso = [[h2 for h2 in text] for text in text1]

target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
test_lists = []
titles = []
for uu, title1 in zip(target_h2_stringso, all_title):
    if len(uu) == 8:
        test_lists.append([i.find_next_sibling().find_all('span', class_='text') for i in uu])
        titles.append(title1)
    else:
        false.append({"企業名":title1,"評価":uu})
        
df_false = pd.DataFrame(false, columns=['title','uu'])
#-------------------------------------------------------------------------ひとまず終了き------------------------------------------------------------------------#
# d_list=[]
sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]
#zfill完了
eval_sales_list = [[convert_sales_figure(sale) for sale in sale_list] for sale_list in sales_list]
#採点完了
uriagedaka_stock = [check_uriagedaka(sale) for sale in eval_sales_list]

#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
eps_lists=[[ep.text for ep in test_list[1]]for test_list in test_lists]
#zfill完了
zeps_list = [[convert_eps(ep) for ep in eps_list]for eps_list in eps_lists]
eps_stock = [check_eps(zeps) for zeps in zeps_list]

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

op_lists=[[op.text for op in test_list[2]]for test_list in test_lists]
zop_list = [[eval(s.split('%')[0]) for s in op_list]for op_list in op_lists] # '15.65' のような文字列を取得
eiri_stock = [check_eiri(zop)for zop in zop_list]


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

operation_lists=[[operation.text for operation in test_list[3]]for test_list in test_lists]
zoption_list = [[floatparcent_hennkan(s) for s in operation_list]for operation_list in operation_lists]
zikosihon_stock = [check_zikosihon(zoption)for zoption in zoption_list]


#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
cash_lists=[[cash.text for cash in test_list[5]]for test_list in test_lists]
  #zfill完了
eval_cash_list = [[convert_sales_figure(cash)for cash in cash_list]for cash_list in cash_lists]
genkin_stock = [check_genkin(eval_cash)for eval_cash in eval_cash_list]
#-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#
cf_flow_lists=[[cfflow.text for cfflow in test_list[4]]for test_list in test_lists]
#zfill完了
eval_cf_flow_list = [[convert_flow_figure(flow)for flow in cf_flow_list]for cf_flow_list in cf_flow_lists]
# モジュール内の関数を呼び出す
cflow_stock = [check_cflow_order(eval_cf)for eval_cf in eval_cf_flow_list]

#-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
#zfill完了
stock_lists=[[stock.text for stock in test_list[6]]for test_list in test_lists]
zstock_list = [[convert_stock_figure(s) for s in stock_list]for stock_list in stock_lists]
hitokabu_stock=[check_zstock_order(zstock)for zstock in zstock_list]

#-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
return_lists=[[return_money.text for return_money in test_list[7]]for test_list in test_lists]
#zfill完了
zreturn_list = [[convert_return(return_money)for return_money in return_list]for return_list in return_lists]
seikou_stock=[check_zreturn_range(zreturn)for zreturn in zreturn_list]
#-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#


analyzes_dict = {}
for i in range(len(titles)):
    analyzes_dict[titles[i]] = [uriagedaka_stock[i], eps_stock[i], eiri_stock[i], zikosihon_stock[i], genkin_stock[i], cflow_stock[i], hitokabu_stock[i],seikou_stock[i]]

gooo = [[key, len(va)] for key, value in analyzes_dict.items() for va in value]
gooo_dict = {}
for lst in gooo:
    key, val = lst
    if key not in gooo_dict:
        gooo_dict[key] = []
    gooo_dict[key].append(val)
for key, value in gooo_dict.items():
    gooo_dict[key] = list(map(lambda x: '○' if x <= 6 else x, value))

ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 4:
        ed_list[key] = value
print(ed_list)
ed_df = pd.DataFrame.from_dict(ed_list, orient='index', columns=['売上高','EPS','営業利益率','自己資本比率','現金','営業キャッシュフロー','一株配当','配当性向'])
ed_df.style.set_properties(**{'text-align': 'center'})
ed_df.to_csv('csv/stock_target.csv',index=True,encoding='utf-8-sig',mode='a')





733
{'8999 グランディハウス': ['○', '○', 16, '○', '○', 7, '○', 9], '6918 アバールデータ': ['○', '○', 8, '○', '○', 8, '○', '○'], '6291 日本エアーテック': ['○', '○', 13, '○', '○', '○', '○', '○']}


In [491]:
ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 2:
        ed_list[key] = value

In [634]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
import pickle
import itertools
import random
#1番目
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

#2番目
#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))

#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_url=[]
false=[]
all_title=[]
text1=[]
for flatclosing_url in flatclosing_urls[30:35]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    all_title.append(title)
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    tagget= soup.find_all('h2')
    text1.append(tagget)
text1_stringso = [[h2 for h2 in text] for text in text1]

target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
test_lists = []
#見た目的にはこの方が綺麗
# for uu, title1 in zip(target_h2_stringso, all_title):
#     if len(uu) == 8:
#         test_lists.append({"企業名": title1, "評価": [i.find_next_sibling().find_all('span', class_='text') for i in uu]})
#     else:
#         false.append({"企業名":title1,"評価":uu})
        
df_false = pd.DataFrame(false, columns=['title','uu'])
#-------------------------------------------------------------------------ひとまず終了き------------------------------------------------------------------------#

# d_list=[]
sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]
#zfill完了
eval_sales_list = [[convert_sales_figure(sale) for sale in sale_list] for sale_list in sales_list]
#採点完了
uriagedaka_stock = [check_uriagedaka(sale) for sale in eval_sales_list]

#-------------------------------------------------------------------------2:ここからEPS2023/01/05--------------------------------------------------------------------------#
eps_lists=[[ep.text for ep in test_list[1]]for test_list in test_lists]
#zfill完了
zeps_list = [[convert_eps(ep) for ep in eps_list]for eps_list in eps_lists]
eps_stock = [check_eps(zeps) for zeps in zeps_list]

#-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

op_lists=[[op.text for op in test_list[2]]for test_list in test_lists]
zop_list = [[eval(s.split('%')[0]) for s in op_list]for op_list in op_lists] # '15.65' のような文字列を取得
eiri_stock = [check_eiri(zop)for zop in zop_list]


#-------------------------------------------------------------------------4:ここから自己資本利益率2023/01/05-------------------------------------------------------------------------#

operation_lists=[[operation.text for operation in test_list[3]]for test_list in test_lists]
zoption_list = [[floatparcent_hennkan(s) for s in operation_list]for operation_list in operation_lists]
zikosihon_stock = [check_zikosihon(zoption)for zoption in zoption_list]


#-------------------------------------------------------------------------5:ここから現金2023/01/05-------------------------------------------------------------------------#
cash_lists=[[cash.text for cash in test_list[5]]for test_list in test_lists]
  #zfill完了
eval_cash_list = [[convert_sales_figure(cash)for cash in cash_list]for cash_list in cash_lists]
genkin_stock = [check_genkin(eval_cash)for eval_cash in eval_cash_list]
#-------------------------------------------------------------------------6:ここからCF -------------------------------------------------------------------------#
cf_flow_lists=[[cfflow.text for cfflow in test_list[4]]for test_list in test_lists]
#zfill完了
eval_cf_flow_list = [[convert_flow_figure(flow)for flow in cf_flow_list]for cf_flow_list in cf_flow_lists]
# モジュール内の関数を呼び出す
cflow_stock = [check_cflow_order(eval_cf)for eval_cf in eval_cf_flow_list]

#-------------------------------------------------------------------------7:ここから一株当たり-------------------------------------------------------------------------#
#zfill完了
stock_lists=[[stock.text for stock in test_list[6]]for test_list in test_lists]
zstock_list = [[convert_stock_figure(s) for s in stock_list]for stock_list in stock_lists]
hitokabu_stock=[check_zstock_order(zstock)for zstock in zstock_list]

#-------------------------------------------------------------------------8:配当性向-------------------------------------------------------------------------#
return_lists=[[return_money.text for return_money in test_list[7]]for test_list in test_lists]
#zfill完了
zreturn_list = [[convert_return(return_money)for return_money in return_list]for return_list in return_lists]
seikou_stock=[check_zreturn_range(zreturn)for zreturn in zreturn_list]
#-------------------------------------------------------------------------ひとまず終了------------------------------------------------------------------------#


analyzes_dict = {}
for i in range(len(all_title)):
    analyzes_dict[all_title[i]] = [uriagedaka_stock[i], eps_stock[i], eiri_stock[i], zikosihon_stock[i], genkin_stock[i], cflow_stock[i], hitokabu_stock[i],seikou_stock[i]]

gooo = [[key, len(va)] for key, value in analyzes_dict.items() for va in value]
gooo_dict = {}
for lst in gooo:
    key, val = lst
    if key not in gooo_dict:
        gooo_dict[key] = []
    gooo_dict[key].append(val)
for key, value in gooo_dict.items():
    gooo_dict[key] = list(map(lambda x: '○' if x <= 4 else x, value))

ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 4:
        ed_list[key] = value
print(ed_list)
ed_df = pd.DataFrame.from_dict(ed_list, orient='index', columns=['売上高','EPS','営業利益率','自己資本比率','現金','営業キャッシュフロー','一株配当','配当性向'])
ed_df.style.set_properties(**{'text-align': 'center'})
ed_df.to_csv('csv/stock_target.csv',index=True,encoding='utf-8-sig',mode='a')






733


KeyError: 0

In [638]:
len(flatclosing_urls)

180

In [479]:
ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 2:
        ed_list[key] = value
print(ed_list)


{'9104 商船三井': [8, '○', 14, 14, '○', 7, '○', 12], '9101 日本郵船': [8, '○', 15, 14, 8, '○', '○', 10], '9110 NSユナイテッド海運': ['○', '○', 12, 12, '○', '○', '○', 9], '1518 三井松島 HD': [9, '○', 14, '○', '○', '○', 9, 8]}


In [623]:
test_lists = []
for uu, title1 in zip(target_h2_stringso, all_title):
    if len(uu) == 8:
        test_lists.append([i.find_next_sibling().find_all('span', class_='text') for i in uu])


In [627]:
len(test_lists)


4

In [462]:
gooo = [[key, len(va)] for key, value in analyzes_dict.items() for va in value]
gooo_dict = {}
for lst in gooo:
    key, val = lst
    if key not in gooo_dict:
        gooo_dict[key] = []
    gooo_dict[key].append(val)
for key, value in gooo_dict.items():
    gooo_dict[key] = list(map(lambda x: '○' if x >= 0 else x, value))


In [466]:
ed_list = {}
for key, value in gooo_dict.items():
    if value.count('○') >= 2:
        ed_list[key] = value
print(ed_list)
import pandas as pd
ed_df = pd.DataFrame.from_dict(ed_list, orient='index', columns=['売上高','EPS','営業利益率','自己資本比率','現金','営業キャッシュフロー','一株配当','配当性向'])


{'9104 商船三井': ['○', '○', '○', '○', '○', '○', '○', '○'], '9101 日本郵船': ['○', '○', '○', '○', '○', '○', '○', '○'], '9110 NSユナイテッド海運': ['○', '○', '○', '○', '○', '○', '○', '○'], '1518 三井松島 HD': ['○', '○', '○', '○', '○', '○', '○', '○'], '9308 乾汽船': ['○', '○', '○', '○', '○', '○', '○', '○']}


In [549]:
ed_df

,売上高,EPS,営業利益率,自己資本比率,現金,営業キャッシュフロー,一株配当,配当性向
3245 ディア・ライフ,○,6,7,○,6,6,○,○
7638 NEW ART HOLDINGS,6,○,9,○,6,○,○,○
2986 LA HD,○,○,○,○,○,○,○,○
6651 日東工業,6,○,11,○,○,8,○,○


In [577]:
for uu, title in zip(target_h2_stringso, all_title):
    if len(uu) == 8:
        test_lists = [[i.find_next_sibling().find_all('span', class_='text') for i in uu]]
    else:
        print(title)







8737 あかつき本社


In [344]:
d_list=[]
for fi in fine:
    if fi.count('○') >= 0:
        d_list.append(fi)
# d_list=[fi for if fi.count('○') >= 0 for alltitle in all_title]

In [345]:
d_list

[['○', '○', '○', '○', '○', '○', '○', '○'],
 ['○', '○', '○', '○', '○', '○', '○', '○'],
 ['○', '○', '○', '○', '○', '○', '○', '○'],
 ['○', '○', '○', '○', '○', '○', '○', '○'],
 ['○', '○', '○', '○', '○', '○', '○', '○']]

In [256]:
focus_url=[]
text=[]
for flatclosing_url in flatclosing_urls[0:3]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    text.append(tagget)


In [11]:
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))
#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
flatclosing_urls = list(itertools.chain(ex_data.values()))
focus_url=[]
text1=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    tagget= soup.find_all('h2')
    text1.append(tagget)
text1_stringso = [[h2 for h2 in text] for text in text1]
        
target_h2_stringso = [[t for t in ext if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or
                      "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t] for ext in text1_stringso ]
test_lists = [[i.find_next_sibling().find_all('span', class_='text') for i in uu]for uu in target_h2_stringso   if len(uu)==8]



# test_lists=[]
# for uu in target_h2_stringso:
#     if len(uu)==8:
#         for i in uu:
#             bro=i.find_next_sibling()
#             ss = bro.find_all('span', class_='text')
#             test_lists.append(ss)
        
        
        
#     for tag in soup.find_all('h2'):
#         text1.append(tag.text)
# ii = []
# for i in range(len(text1)):
#     ii.append([html.fromstring(t) for t in text1[i]])
# ii=[]
# for tem in text1:
#     for kkc in tem:
#         ii.append(kkc)



733


NameError: name 'sleep' is not defined

In [258]:
import pandas as pd
df = pd.DataFrame(all_title,columns=["会社名"])
df["企業コード"] = df["会社名"].str.extract("(\d+)", expand=True)
df["会社名"] = df["会社名"].str.extract(" (.*)", expand=True)
for i, uriages in enumerate(uriagedaka_stock):
    df.loc[i, "売上"] = len(uriages)
df['売上'] = df['売上'].astype(int)

for i, epssto in enumerate(eps_stock):
    df.loc[i, "EPS"] = len(epssto)
df['EPS'] = df['EPS'].astype(int)

for i, zikosihon in enumerate(zikosihon_stock):
    df.loc[i, "自己資本"] = len(zikosihon)
df['自己資本'] = df['自己資本'].astype(int)

for i, genkin in enumerate(genkin_stock):
    df.loc[i, "現金"] = len(genkin)
df['現金'] = df['現金'].astype(int)

for i, cflo in enumerate(cflow_stock):
    df.loc[i, "営業キャッシュフロー"] = len(cflo)
df['営業キャッシュフロー'] = df['営業キャッシュフロー'].astype(int)

for i, hitokabu in enumerate(hitokabu_stock):
    df.loc[i, "一株配当"] = len(hitokabu)
df['一株配当'] = df['一株配当'].astype(int)

for i, seiko in enumerate(seikou_stock):
    df.loc[i, "配当性向"] = len(seiko)
df['配当性向'] = df['配当性向'].astype(int)

print(df)


#     analyze.append(hitokabu_stock)
#     analyze.append(seikou_stock)


          会社名 企業コード  売上  EPS  自己資本  現金  営業キャッシュフロー  一株配当  配当性向
0        商船三井  9104   8    5    14   6           7     5    12
1        日本郵船  9101   8    4    14   8           6     5    10
2  NSユナイテッド海運  9110   5    6    12   3           6     5     9
3     三井松島 HD  1518   9    5     2   6           6     9     8
4         乾汽船  9308   8    8     8   7           6    10    10


In [238]:
for uriages in uriagedaka_stock:
    print(len(uriages))

8
8
5
9
8


In [234]:
len_not_uriage =[[print(len(uriage))for uriage in uriages]for uriages in uriagedaka_stock]


SyntaxError: invalid syntax (3231367424.py, line 1)

In [29]:
d_list=[]
for test_list in test_lists:
    if len(test_list)==8:
            sales_list=[[sale.text for sale in test_list[0]]for test_list in test_lists]
            eval_sales_list = [[convert_sales_figure(sale) for sale in sale_list] for sale_list in sales_list]
            uriagedaka_stock = [check_uriagedaka(sale) for sale in eval_sale_list]
            
            eps_lists=[[ep.text for ep in test_list[1]]for test_list in test_lists]
            #zfill完了
            zeps_list = [[convert_eps(ep) for ep in eps_list]for eps_list in eps_lists]
            eps_stock = [check_eps(zeps) for zeps in zeps_list]

        #-------------------------------------------------------------------------3:ここから営業利益率2023/01/05- -------------------------------------------------------------------------#     

            op_lists=[[op.text for op in test_list[2]]for test_list in test_lists]
            zop_list = [[eval(s.split('%')[0]) for s in op_list]for op_list in op_lists]
            eiri_stock = [check_eiri(zop)for zop in zop_list]
        



In [1272]:
text1_stringso = [[h2.text for h2 in text] for text in text1]


In [33]:
eps_stock

[['NOT (at index 2)',
  'NOT (at index 3)',
  'NOT (at index 5)',
  'NOT (at index 6)',
  'NOT (at index 8)'],
 ['NOT (at index 2)',
  'NOT (at index 6)',
  'NOT (at index 7)',
  'NOT (at index 9)']]

In [28]:
uriagedaka_stock[0]

['NOT (at index 1)',
 'NOT (at index 2)',
 'NOT (at index 4)',
 'NOT (at index 8)',
 'NOT (at index 9)',
 'NOT (at index 11)',
 'NOT (at index 12)',
 'NOT (at index 13)']

In [1205]:
focus_url=[]
text1=[]
for flatclosing_url in flatclosing_urls[:2]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    decopo=soup.find_all('a')
    for a in decopo:
        a.decompose()
    for tag in soup.find_all('h2'):
        text1.append(tag.text).    flatclosing_urlsの１と2のそれぞれをtext1[0],text1[1]に入れたtext1にしたいのですが
​
for tem in ii:
    print(type(tem))
# target_h2 = []
# for t in ii:
#     if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
#         target_h2.append(t)

In [1206]:
print(type(tem))

<class 'bs4.element.ResultSet'>


In [1207]:
ii[0]

IndexError: list index out of range

In [296]:

for tem in analyze[2]:
    print(tem)
    print(type(tem))

['NOT (at index 3)', 'NOT (at index 6)', 'NOT (at index 7)', 'NOT (at index 10)', 'NOT (at index 11)']
<class 'list'>
['NOT (at index 2)', 'NOT (at index 3)', 'NOT (at index 5)', 'NOT (at index 6)', 'NOT (at index 7)', 'NOT (at index 10)']
<class 'list'>
['NOT (at index 0)', 'NOT (at index 1)', 'NOT (at index 2)', 'NOT (at index 3)', 'NOT (at index 4)', 'NOT (at index 5)', 'NOT (at index 6)', 'NOT (at index 7)', 'NOT (at index 8)', 'NOT (at index 9)', 'NOT (at index 10)', 'NOT (at index 11)']
<class 'list'>
['NOT (at index 1)', 'NOT (at index 3)', 'NOT (at index 4)', 'NOT (at index 5)', 'NOT (at index 6)', 'NOT (at index 7)', 'NOT (at index 8)', 'NOT (at index 9)', 'NOT (at index 10)', 'NOT (at index 11)', 'NOT (at index 12)', 'NOT (at index 13)']
<class 'list'>
['NOT (at index 8)', 'NOT (at index 11)', 'NOT (at index 12)']
<class 'list'>
['NOT (at index 1)', 'NOT (at index 4)', 'NOT (at index 8)', 'NOT (at index 9)', 'NOT (at index 10)', 'NOT (at index 12)']
<class 'list'>
['NOT (at i

In [1106]:
text = []#正規は８項目  
for element in list_tagget:
    for i in range(len(element)):
        ele = element[i]
        for a in ele.find_all('a'):
            a.decompose()
            text.append(element)

        


In [1111]:
focus_url=[]
list_tagget=[]
for flatclosing_url in flatclosing_urls[0:3]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(random.uniform(3, 8)) 
    focus_url.append(url)
    r=requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    list_tagget.append(tagget)

In [1139]:
tagget[0]

<h2>売上高</h2>

In [660]:
with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(ex_data, f)
flatclosing_urls = list(itertools.chain(ex_data.values()))

In [335]:
with open('irbank/csv/securities_codes.pickle','rb') as f:
    securities_codes=pickle.load(f)

In [78]:
def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)

In [77]:
def convert_sales(*args):
    eval_list = []
    for arg in args:
        eval_arg = []
        for value in arg:
            # 兆しかない時
            if '兆' in value and '億' not in value and '万' not in value:
                hennkan = value.replace('兆', '*10**12')
                eval_arg.append(eval(hennkan))
        eval_list.append(eval_arg)
    return eval_list


In [46]:


def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(random.uniform(3, 8)) 
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    return securities_codes





def float_hennkan(s):
    # *だけの時
    if '*' in s:
        s = s.split('円')[0]  # '15.65' のような文字列を取得
        s = s.replace('*', '')  # * を削除
        if '.'in s:
            integer_part, decimal_part =s.split('.')  # '15' と '65' のような文字列を取得
            integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
            decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
            hennkan_result= f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
    #小数点の時
    elif '.' in s :
        number_str = s.split('円')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result
      

    else:
        # 整数部と小数部が分離されていない場合
        number_str = s.split('円')[0]  # '86' のような文字列を取得
        integer_part = number_str.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
    

#     if '.' in s:
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result
#     *だけの時
#     elif '*' in s:
#         kn = s.split('*')[1]
#         number_str1 = kn .split('円')[0]  # '86' のような文字列を取得
#         integer_part1 = number_str1.zfill(2)  # 整数部を 0 で埋める
#         hennkan_result = f"{integer_part1}.00"
#         return hennkan_result



def convert_sales_figure(sales_figure: str) -> int:
    #兆しかない時
    if '兆' in sales_figure and '億' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12')
        return eval(hennkan)
    #億しかない時
    elif '億' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8')
        return eval(hennkan)
    #万しかない時
    elif '万' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('万', '*10**4')
        return eval(hennkan)
    #兆と億の時
    elif'兆'in sales_figure and '億' in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12+').replace('億','*10**8')
        return eval(hennkan)
    #億と万の時
    elif'億'in sales_figure and '万' in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8+').replace('万','*10**4')
        return eval(hennkan)
    else:
        hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sales_figure)
        return eval(hennkan)



def convert_eps(eps_figure: str) -> int:
    number_str = eps_figure.split('円')[0]
    hennkan = number_str + '*100'
    return eval(hennkan)


def convert_stock_figure(stock_figure: str) -> int:
    if '.' in stock_figure and '*' not in stock_figure :
        number_str = stock_figure.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_figure and '.' not in stock_figure :
        kn = stock_figure.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    
    elif'*'in stock_figure and '.' in stock_figure:
        kn = stock_figure.split('*')[1]
        number_str = kn.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    else:
        number_str = stock_figure.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"
    

def convert_stock(stock_price: str) -> str:
    if '.' in stock_price:
        number_str = stock_price.split('円')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"
    elif '*' in stock_price:
        kn = stock_price.split('*')[1]
        number_str1 = kn .split('円')[0]
        integer_part1 = number_str1.zfill(2)
        return f"{integer_part1}.00"
    else:
        number_str = stock_price.split('円')[0]
        integer_part = number_str.zfill(2)
        return f"{integer_part}.00"



def convert_return(return_figure: str) -> str:
    if '.' in return_figure:
        number_str = return_figure.split('%')[0]
        integer_part, decimal_part = number_str.split('.')
        integer_part = integer_part.zfill(2)
        decimal_part = decimal_part.zfill(2)
        return f"{integer_part}.{decimal_part}"

    else:
        integer_part = return_figure.split('%')[0]
        integer_part = integer_part.zfill(2)
        return f"{integer_part}.00"

def process_cashes(cashes):
    cash_list = [cash.text for cash in cashes]
    eval_cash_list = []
    for cash in cash_list:
        # 兆しかない時
        if '兆' in cash and '億' not in cash and '万' not in cash:
            hennkan = cash.replace('兆', '*10**12')
            eval_cash_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('億', '*10**8')
            eval_cash_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in cash and '兆' not in cash and '万' not in cash:
            hennkan = cash.replace('万', '*10**4')
            eval_cash_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in cash and '億' in cash:
            hennkan = cash.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cash_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in cash and '万' in cash:
            hennkan = cash.replace('億', '*10**8+').replace('万','*10**4')
            eval_cash_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cash_list.append(eval(hennkan))
    return eval_cash_list


import re

def process_cfflows(cfflows):
    cf_flow_list = [cfflow.text for cfflow in cfflows]
    eval_cf_flow_list = []
    for flow in cf_flow_list:
        # 兆しかない時
        if '兆' in flow and '億' not in flow and '万' not in flow:
            hennkan = flow.replace('兆', '*10**12')
            eval_cf_flow_list.append(eval(hennkan))
        # 億しかない時
        elif '億' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('億', '*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 万しかない時
        elif '万' in flow and '兆' not in flow and '万' not in flow:
            hennkan = flow.replace('万', '*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        # 兆と億の時
        elif '兆' in flow and '億' in flow:
            hennkan = flow.replace('兆', '*10**12+').replace('億','*10**8')
            eval_cf_flow_list.append(eval(hennkan))
        # 億と万の時
        elif '億' in flow and '万' in flow:
            hennkan = flow.replace('億', '*10**8+').replace('万','*10**4')
            eval_cf_flow_list.append(eval(hennkan))
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sale)
            eval_cf_flow_list.append(eval(hennkan))
    return eval_cf_flow_list


def convert_japanese_financial_string(string):
    hennkan = string.replace('兆', '*10**12').replace('億', '*10**8').replace('万', '*10**4')
    return eval(hennkan)



def convert_flow_figure(flow_figure: str) -> int:
        #兆しかない時
        if '兆' in flow_figure and '億' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12')
            return eval(hennkan)         
        #億しかない時
        elif '億' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8')
            return eval(hennkan)             
        #万しかない時
        elif '万' in flow_figure and '兆' not in flow_figure and '万' not in flow_figure:
            hennkan = flow_figure.replace('万', '*10**4')
            return eval(hennkan)
        #兆と億の時
        elif'兆'in flow_figure and '億' in flow_figure:
            hennkan = flow_figure.replace('兆', '*10**12+').replace('億','*10**8')
            return eval(hennkan)
        #億と万の時
        elif'億'in flow_figure and '万' in flow_figure:
            hennkan = flow_figure.replace('億', '*10**8+').replace('万','*10**4')
            return eval(hennkan)
        else:
            hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", flow_figure)
            return eval(hennkan)


def floatparcent_hennkan(s):
    if '.' in s:
        number_str = s.split('%')[0]  # '15.65' のような文字列を取得
        integer_part, decimal_part = number_str.split('.')  # '15' と '65' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        decimal_part = decimal_part.zfill(2)  # 小数部を 0 で埋める
        hennkan_result = f"{integer_part}.{decimal_part}"  # 埋めた数値を結合
        return hennkan_result

    else:
        # 整数部と小数部が分離されていない場合
        integer_part = s.split('%')[0]  # '86' のような文字列を取得
        integer_part = integer_part.zfill(2)  # 整数部を 0 で埋める
        hennkan_result = f"{integer_part}.00"
        return hennkan_result
def check_uriagedaka(eval_sale_list):
    result = all(int(eval_sale_list[i]) < int(eval_sale_list[i+1]) for i in range(len(eval_sale_list)-1))
    uriagedaka_stock = []
    if result:
        uriagedaka_stock.append('OK')
    else:
        for i in range(len(eval_sale_list)-1):
            if int(eval_sale_list[i]) >= int(eval_sale_list[i+1]):
                uriagedaka_stock.append(f'NOT (at index {i+1})')
    return uriagedaka_stock


def check_eps(zeps_list):
    result = all(int(zeps_list[i]) < int(zeps_list[i+1]) for i in range(len(zeps_list)-1))
    eps_stock = []
    if result:
        eps_stock.append('OK')
    else:
        for i in range(len(zeps_list)-1):
            if int(zeps_list[i]) >= int(zeps_list[i+1]):
                eps_stock.append(f'NOT (at index {i+1})')
    return eps_stock

def check_eiri(zop_list):
    zop_list = [float(x) for x in zop_list]
    resault = all(10 <= float(zop_list[i]) for i in range(len(zop_list)))
    eiri_stock = []
    if resault:
        eiri_stock.append('OK')
    else:
        for i in range(len(zop_list)):
            if 10 >= float(zop_list[i]):
                eiri_stock.append(f'NOT (at index {i})')
    return eiri_stock

def check_zikosihon(zoption_list):
    oolist = [float(x) for x in zoption_list]
    result = all(40 <= float(oolist[i]) for i in range(len(oolist)))
    zikosihon_stock = []
    if result:
        zikosihon_stock.append('OK')
    else:
        for i in range(len(oolist)):
            if 40 >= float(oolist[i]):
                zikosihon_stock.append(f'NOT (at index {i})')
    return zikosihon_stock


def check_genkin(eval_cash_list):
    result = all(int(eval_cash_list[i]) < int(eval_cash_list[i+1]) for i in range(len(eval_cash_list)-1))
    genkin_stock = []
    if result:
        genkin_stock.append('OK')
    else:
        for i in range(len(eval_cash_list)-1):
            if int(eval_cash_list[i]) >= int(eval_cash_list[i+1]):
                genkin_stock.append(f'NOT (at index {i+1})')
    return genkin_stock


def check_cflow_order(eval_cf_flow_list):
    result = all(int(eval_cf_flow_list[i]) < int(eval_cf_flow_list[i+1]) for i in range(len(eval_cf_flow_list)-1))
    cflow_stock=[]
    if result:
        cflow_stock.append('OK')
    else:
        for i in range(len(eval_cf_flow_list)-1):
            if int(eval_cf_flow_list[i]) >= int(eval_cf_flow_list[i+1]):
                cflow_stock.append(f'NOT (at index {i+1})')
    return cflow_stock

def check_zstock_order(zstock_list):
    result = all(float(zstock_list[i]) < float(zstock_list[i+1]) for i in range(len(zstock_list)-1))
    hitokabu_stock=[]
    if result:
        hitokabu_stock.append('OK')
    else:
        for i in range(len(zstock_list)-1):
            if float(zstock_list[i]) >= float(zstock_list[i+1]):
                hitokabu_stock.append(f'NOT (at index {i+1})')
    return hitokabu_stock


def check_zreturn_range(zreturn_list):
    int_zreturn_list = [float(x) for x in zreturn_list]
    result = all(30<int(int_zreturn_list[i]<50) for i in range(len(int_zreturn_list)))
    seikou_stock=[]
    if result:
        seikou_stock.append('OK')
    else:
        for i in range(len(int_zreturn_list)):
            if int(30>int_zreturn_list[i] or int_zreturn_list[i]>70):
                seikou_stock.append(f'NOT (at index {i})')
    return seikou_stock



def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link

def convert_sales_figure(sales_figure: str) -> int:
    #兆しかない時
    if '兆' in sales_figure and '億' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12')
        return eval(hennkan)
    #億しかない時
    elif '億' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8')
        return eval(hennkan)
    #万しかない時
    elif '万' in sales_figure and '兆' not in sales_figure and '万' not in sales_figure:
        hennkan = sales_figure.replace('万', '*10**4')
        return eval(hennkan)
    #兆と億の時
    elif'兆'in sales_figure and '億' in sales_figure:
        hennkan = sales_figure.replace('兆', '*10**12+').replace('億','*10**8')
        return eval(hennkan)
    #億と万の時
    elif'億'in sales_figure and '万' in sales_figure:
        hennkan = sales_figure.replace('億', '*10**8+').replace('万','*10**4')
        return eval(hennkan)
    else:
        hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", sales_figure)
        return eval(hennkan)

In [ ]:
test__lists

In [155]:
import requests
from cachecontrol import CacheControl 
from cachecontrol.caches import FileCache

session = requests.Session()
# sessionをラップしたcached_sessionを作る。
# キャッシュはファイルとして .webcache ディレクトリ内に保存する。
cached_session = CacheControl(session, cache=FileCache('.webcache'))

response = cached_session.get('https://irbank.net/E31167/results') 

# response.from_cache属性でキャッシュから取得されたレスポンスかどうかを取得できる。
print(f'from_cache: {response.from_cache}') 
print(f'status_code: {response.status_code}')

from_cache: False
status_code: 200


NameError: name 'number_str' is not defined

In [180]:
from logging import getLogger,StreamHandler,DEBUG,ERROR

logger=getLogger(__name__)
handler=StreamHandler()
handler.setLevel(ERROR)
logger.setLevel(DEBUG)
logger.addHandler(handler)
logger.debug('これはデバッグログ')

これはデバッグログ
これはデバッグログ
これはデバッグログ
これはデバッグログ
これはデバッグログ


In [195]:
securities_codes = get_securities_codes(27,28)
for x in securities_codes:
    with open('irbank/csv/securities_codes.pickle','wb') as f:
        pickle.dump(x,f)
    with open('irbank/csv/securities_codes.pickle','rb') as f:
        x=pickle.load(f)
print(x)

5019


time: 0.000744820 [sec]
time: 0.000028849 [sec]
time: 0.000043154 [sec]
time: 0.000023127 [sec]


In [477]:
 df=pd.DataFrame(dd_list)
df.head()
df.shape

(40, 4)

Error: iterable expected, not int

Error: iterable expected, not int

In [764]:
import itertools
with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

729


In [797]:
existing_dat

NameError: name 'existing_dat' is not defined

In [599]:
with open('irbank/csv/code.pickle', 'rb') as f:
            company_code = pickle.load(f)

In [619]:
del existing_data[0]

709


In [3]:
import time

# 関数の実行時間を測るデコレータ
def tictoc(func):
    def _wrapper(*args, **keywargs):
        start_time = time.time()
        result = func(*args, **keywargs)
        print('time: {:.9f} [sec]'.format(time.time() - start_time))
        return result
    return _wrapper

In [4]:
from functools import cache

@tictoc
@cache  # デコレータを付与
def func(num):
    # なんらかの重い処理
    for _ in range(1000000):
        num += 1
    return num

# 1度目の呼び出し
func(1)

# 2度目の呼び出し
func(1)

time: 0.077428818 [sec]
time: 0.000001907 [sec]


1000001

In [207]:
func(1)

time: 0.000001907 [sec]


1000001

In [5]:
#1
import random
@tictoc
@cache
def get_securities_codes(start_page, end_page):
    url='https://minkabu.jp/financial_item_ranking/dividend_yield?page={}'
    dd_list=[]
    for i in range(start_page, end_page+1):
        target_url=url.format(i)
        r=requests.get(target_url)
        sleep(random.uniform(3, 8)) 
        soup=BeautifulSoup(r.text,'html.parser')
        tab=soup.find('table',class_='md_table rnk_li')
        conts=tab.find_all('tr')
        del conts[0]

        for cont in conts:
            rank=cont.find('span',class_='ranking_no fsize_l').text
            code=cont.find('div',class_='md_sub').text
            name=cont.find('div',class_='fwb w90p').text
            y=cont.find('td',class_='tar cur vamd').text
            yeild= y.strip().replace('%', '')
            dic={
                'ランキング':rank,
                '証券コード':code,
                '企業名':name,
                '配当利回り':yeild,
            }
            
            
            if float(yeild) >= 3.75:
                dd_list.append(dic)
    securities_codes = [d['証券コード'] for d in dd_list]
    #csvに保存まだ更新はできず追加しかされない
    new_data=pd.DataFrame(dd_list)
    if os.path.exists('irbank/csv/20230115ranking.csv'):
                df=pd.read_csv('irbank/csv/20230115ranking.csv')
                df=pd.concat([df, new_data], ignore_index=True)
                
    else:
        df=new_data
    df.to_csv('irbank/csv/20230115ranking.csv',index=None,encoding='utf-8-sig')

    return securities_codes


In [798]:
#1

result = {}
#1ページ目から取得なので値に0はいらない
for page in range(30, 37, 1):
    securities_codes = get_securities_codes(page, page)
    result[page] = securities_codes
    if os.path.exists('irbank/csv/securities_codes.pickle'):
     # ファイルから既存のデータを読み込む
        with open('irbank/csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
        # 既存のデータを更新する
        existing_data.update(result)
        # ファイルに追加されたデータを保存する
        with open('irbank/csv/securities_codes.pickle', 'wb') as f:
            pickle.dump(existing_data, f)
    else:
        del result[0]
        with open('irbank/csv/securities_codes.pickle', 'wb') as f:
            pickle.dump(result, f)

time: 5.357740641 [sec]
time: 6.625604153 [sec]
time: 8.049380302 [sec]
time: 7.945807934 [sec]
time: 5.542266130 [sec]
time: 5.170459032 [sec]
time: 6.947769880 [sec]


In [61]:
import itertools
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
            del existing_data[0]
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

713


In [543]:
#     with open('irbank/csv/ranking.csv', 'w', newline='') as csvfile:
#         fieldnames = ['ランキング','証券コード','企業名','配当利回り']
#         #辞書型を書き込むためのクラス
#         writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#         writer.writeheader()
#         for d in dd_list:
#             writer.writerow({'ランキング':d['ランキング'],'証券コード': d['証券コード'],'企業名':d['企業名'],'配当利回り':d['配当利回り']})
    


In [629]:
closing_urls= []
# for code in securities_codes[:3]:
for code in flattened_codes[0:1]:
    closing_urls.append(get_closing_url(code))
print (closing_urls)

time: 0.000003099 [sec]
['/E04236/results']


In [30]:
@tictoc
@cache
def get_closing_url(code):
    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(random.uniform(3, 8)) 
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select_one('div.csb.cc2 > ul:nth-of-type(2) > li:first-of-type')
    taget = element.find('a')
    link = taget.get('href')
    return link


In [777]:
#2
#ここからclosing_dict=
import pickle
closing_dict = {}
# for page in range(0, 100, 10):
#     # your code here
#     time.sleep(60) 
for code in flattened_codes[1:300]:
    new_url = get_closing_url(code)
    closing_dict[code] = new_url

# 既存のpickleファイルがある場合
    if os.path.exists('irbank/csv/closing_dict.pickle'):
        # ファイルから既存のデータを読み込む
        with open('irbank/csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)
        # 既存のデータに新しいデータを追加
        ex_data.update(closing_dict)
        # ファイルに追加されたデータを保存する
        with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(ex_data, f)
    # 初めての保存の場合
    else:
        with open('irbank/csv/closing_dict.pickle', 'wb') as f:
            pickle.dump(closing_dict, f)
            
            


time: 2.522117853 [sec]
time: 2.484273195 [sec]
time: 2.296419144 [sec]
time: 2.385443687 [sec]
time: 2.317987204 [sec]
time: 2.381206036 [sec]
time: 2.426397085 [sec]
time: 2.504644871 [sec]
time: 2.421528101 [sec]
time: 2.425532103 [sec]
time: 2.524970055 [sec]
time: 2.401628971 [sec]
time: 2.468930960 [sec]
time: 2.386493921 [sec]
time: 2.442391872 [sec]
time: 2.417729139 [sec]
time: 2.303143263 [sec]
time: 2.543843031 [sec]
time: 2.398165941 [sec]
time: 2.271637201 [sec]


In [778]:
#2
with open('irbank/csv/closing_dict.pickle', 'rb') as f:
            ex_data = pickle.load(f)

In [779]:
len(ex_data)

180

729


In [719]:
#3

#ここからfinal_dict
import pickle
final_dict = {}
focus_url=[]
for flatclosing_url in flatclosing_urls[8:10]:
    url='https://irbank.net{}'.format(flatclosing_url)
    sleep(2)
    focus_url.append(url)
    final_dict[flatclosing_url] = focus_url[-1]
    save_pickle(final_dict, 'irbank/csv/final_dict.pickle')


In [720]:
with open('irbank/csv/final_dict.pickle', 'rb') as f:
    ex_data = pickle.load(f)
final_code = list(itertools.chain(ex_data.values()))


In [721]:
final_code

['https://irbank.net/E04236/results',
 'https://irbank.net/E04235/results',
 'https://irbank.nethttps://irbank.net/E04236/results',
 'https://irbank.nethttps://irbank.net/E04235/results']

In [692]:
#3
with open('irbank/csv/final_dict.pickle', 'rb') as f:
    ex_data = pickle.load(f)
final_code = list(itertools.chain(ex_data.values()))

for i in final_code:
    r=requests.get(i)
    soup = BeautifulSoup(r.text, 'html.parser')
    title=soup.find('h1').text
    tagget= soup.find_all('h2')
    
text=[]
for element in tagget:
    for a in element.find_all('a'):
        a.decompose()
    text.append(element)
target_h2 = []#正規は８項目
for t in text:
    if "売上高" in t or"EPS" in t or "営業利益率" in t or "自己資本比率" in t or "営業活動によるCF" in t or "現金等" in t or "一株配当" in t or "配当性向" in t:
        target_h2.append(t)
test_lists=[]
if len(target_h2)==8:
    for i in target_h2:
        bro=i.find_next_sibling()
        ss = bro.find_all('span', class_='text')
        test_lists.append(ss)



In [696]:
tagge

NameError: name 'tagge' is not defined

In [680]:
nw_url

['https://irbank.net/E04236/results', 'https://irbank.net/E04235/results']

In [671]:
import os
import pickle

def save_pickle(data, filepath):
    # 既存のpickleファイルがある場合
    if os.path.exists(filepath):
        # ファイルから既存のデータを読み込む
        with open(filepath, 'rb') as f:
            ex_data = pickle.load(f)
        # 既存のデータに新しいデータを追加
        ex_data.update(data)
        # ファイルに追加されたデータを保存する
        with open(filepath, 'wb') as f:
            pickle.dump(ex_data, f)
    # 初めての保存の場合
    else:
        with open(filepath, 'wb') as f:
            pickle.dump(data, f)


In [539]:
def convert_yeild_figure(yeild_figure: str) -> int:
    #兆しかない時
    if '兆' in yeild_figure and '億' not in yeild_figure and '万' not in yeild_figure:
        hennkan = yeild_figure.replace('兆', '*10**12')
        return eval(hennkan)
    #億しかない時
    elif '億' in yeild_figure and '兆' not in yeild_figure and '万' not in yeild_figure:
        hennkan = yeild_figure.replace('億', '*10**8')
        return eval(hennkan)
    #万しかない時
    elif '万' in yeild_figure and '兆' not in yeild_figure and '万' not in yeild_figure:
        hennkan = yeild_figure.replace('万', '*10**4')
        return eval(hennkan)
    elif '-' in yeild_figure and '兆' not in yeild_figure and '億' not in yeild_figure and '万' not in yeild_figure:
        hennkan=yeild_figure.replace('-','0')
        return eval(hennkan)
    #兆と億の時
    elif'兆'in yeild_figure and '億' in yeild_figure:
        hennkan = yeild_figure.replace('兆', '*10**12+').replace('億','*10**8')
        return eval(hennkan)
    #億と万の時
    elif'億'in yeild_figure and '万' in yeild_figure:
        hennkan = yeild_figure.replace('億', '*10**8+').replace('万','*10**4')
        return eval(hennkan)
    
    else:
        hennkan = re.sub(r"(兆)|(億)|(万)", lambda x: "*10**12+" if x.group(1) else "*10**8" if x.group(2) else "*10**4", yeild_figure)
        return eval(hennkan)
    
def check_yeild(eval_yeild_lists):
    result = all(int(eval_yeild_lists[i]) < int(eval_yeild_lists[i+1]) for i in range(len(eval_yeild_lists)-1))
    yeild_stock = []
    if result:
        yeild_stock.append('OK')
    else:
        for i in range(len(eval_yeild_lists)-1):
            if int(eval_yeild_lists[i]) >= int(eval_yeild_lists[i+1]):
                yeild_stock.append(f'NOT (at index {i+1})')
    return yeild_stock


1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [645]:
from bs4 import BeautifulSoup
import requests 
import re
from time import sleep
import pandas as pd
import time
from tqdm import tqdm
import pickle
import itertools
import random
with open('csv/securities_codes.pickle', 'rb') as f:
            existing_data = pickle.load(f)
#リストを一緒くたに一つに
flattened_codes = list(itertools.chain(*existing_data.values()))
print(len(flattened_codes))

#flattened_codesの値を渡すとその企業コードのIRBANKのトップページが開き、その中の決算情報ページのリンクURLの一部を取得
with open('csv/yeild_dict.pickle', 'rb') as f:
            ye_data = pickle.load(f)
flatyeild_urls = list(itertools.chain(ye_data.values()))

#-------------------------------------------------------------------------一先ず終了2023/01/05----------------------------------------------------------------------#
#上記を満たした企業コードの決算ページxxxxxx/resultsにアクセスし企業名と条件の財務の値を取得
focus_urls1=[]
text2=[]
all_title1=[]
# for flatyeild_url in flatyeild_urls[10:12]:
#     url='https://irbank.net{}'.format(flatyeild_url)
url='https://irbank.net/E04425/dividend'
sleep(random.uniform(3, 8)) 
focus_urls1.append(url)
for focus_url1 in focus_urls1:
    r1=requests.get(focus_url1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    title1=soup1.find('h1').text
    all_title1.append(title1)
    suii=soup1.find(id='g_3')
    numv=suii.find_all('span', class_='text')
    text2.append(numv)
yeild_lists=[[text2_list.text for text2_list in text2_lists]for text2_lists in text2]
    #zfill完了
eval_yeild_lists = [[convert_yeild_figure(yeild) for yeild in yeild_list] for yeild_list in yeild_lists]
yeild_stock = [check_yeild(yeild) for yeild in eval_yeild_lists]


yeild_analyze_dict = {}
if len(all_title1) == len(yeild_stock):
    for i in range(len(all_title1)):
        yeild_analyze_dict[all_title1[i]] = [yeild_stock[i]]
    goood = [[key, len(vi)] for key, value in yeild_analyze_dict.items() for vi in value]
else:
    print("The length of two lists are different.")
goood_dict = {}
for elst in goood:
    key, val = elst
    if key not in goood_dict:
        goood_dict[key] = []
    goood_dict[key].append(val)
for key, value in goood_dict.items():
    goood_dict[key] = list(map(lambda x: '○' if x <= 2 else x, value))
    
edd_list = {}
for key, value in goood_dict.items():
    if value.count('○') == 1:
        edd_list[key] = value
print(edd_list)
edd_df = pd.DataFrame.from_dict(edd_list, orient='index', columns=['剰余金の配当/配当金の支払額'])
edd_df.style.set_properties(**{'text-align': 'center'})
edd_df.to_csv('csv/yeild_rize.csv',index=True,encoding='utf-8-sig',mode='a')



{'9433 KDDI': ['○']}


In [595]:
goood_dict



{'9104 商船三井': [8]}

In [87]:
focus_url1

['https://irbank.net/E04236/dividend',
 'https://irbank.net/E04235/dividend',
 'https://irbank.net/E04239/dividend']

In [49]:
#ここから配当推移ページ
@tictoc
@cache
def get_yeild_curve(code):

    url = f'https://irbank.net/{code}/ir'
    r = requests.get(url)
    sleep(random.uniform(3, 8)) 
    soup = BeautifulSoup(r.text,'html.parser')
    element = soup.select('div.csb.cc2 > ul:nth-of-type(2) > li:nth-of-type(7)')
    for ec in element:
        taget = ec.find('a')
        link = taget.get('href')
    return link

In [59]:
#2
#ここから配当推移ページ
import pickle
import os
yeild_dict = {}
# for page in range(0, 100, 10):
#     # your code here
#     time.sleep(60) 
for code in flattened_codes[110:140]:
    yeuld_url = get_yeild_curve(code)
    yeild_dict[code] = yeuld_url

# 既存のpickleファイルがある場合
    if os.path.exists('csv/yeild_dict.pickle'):
        # ファイルから既存のデータを読み込む
        with open('csv/yeild_dict.pickle', 'rb') as f:
            ye_data = pickle.load(f)
        # 既存のデータに新しいデータを追加
        ye_data.update(yeild_dict)
        # ファイルに追加されたデータを保存する
        with open('csv/yeild_dict.pickle', 'wb') as f:
            pickle.dump(ye_data, f)
    # 初めての保存の場合
    else:
        with open('csv/yeild_dict.pickle', 'wb') as f:
            pickle.dump(yeild_dict, f)
            
            

time: 0.000003099 [sec]
time: 0.000004292 [sec]
time: 0.000003815 [sec]
time: 0.000003815 [sec]
time: 0.000005245 [sec]
time: 0.000006199 [sec]
time: 0.000003099 [sec]
time: 0.000005007 [sec]
time: 0.000041962 [sec]
time: 0.000007153 [sec]
time: 0.000005960 [sec]
time: 0.000005007 [sec]
time: 0.000003815 [sec]
time: 0.000006914 [sec]
time: 0.000005007 [sec]
time: 3.956977129 [sec]
time: 5.480154037 [sec]
time: 4.167751074 [sec]
time: 5.064061880 [sec]
time: 5.905288935 [sec]
time: 4.087606907 [sec]
time: 7.761102200 [sec]
time: 6.688001156 [sec]
time: 5.672971010 [sec]
time: 6.611789942 [sec]
time: 3.800131083 [sec]
time: 7.404054880 [sec]
time: 7.714793921 [sec]
time: 4.473711014 [sec]
time: 3.784794807 [sec]
